In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")


In [4]:
# cargamos el dataframe correspondiente 
df = pd.read_csv("archivos/christmas_movies_01.csv")

display(df.head())

,title,rating,runtime,imdb_rating,meta_score,genre,release_year,description,director,stars,votes,gross,img_src,type
0,Love Actually,R,135.0,7.6,55.0,"Comedy, Drama, Romance",2003.0,Follows the lives of eight very different coup...,Richard Curtis,"Hugh Grant, Martine McCutcheon, Liam Neeson, L...","517,283",$59.70M,https://m.media-amazon.com/images/M/MV5BNThkNj...,Movie
1,Home Alone,PG,103.0,7.7,63.0,"Comedy, Family",1990.0,"An eight-year-old troublemaker, mistakenly lef...",Chris Columbus,"Macaulay Culkin, Joe Pesci, Daniel Stern, John...","629,713",$285.76M,https://m.media-amazon.com/images/M/MV5BMzFkM2...,Movie
2,National Lampoon's Christmas Vacation,PG-13,97.0,7.5,49.0,Comedy,1989.0,The Griswold family's plans for a big family C...,Jeremiah S. Chechik,"Chevy Chase, Beverly D'Angelo, Juliette Lewis,...","213,196",$71.32M,https://m.media-amazon.com/images/M/MV5BMGZkMW...,Movie
3,Elf,PG,97.0,7.1,66.0,"Adventure, Comedy, Family",2003.0,"Raised as an oversized elf, Buddy travels from...",Jon Favreau,"Will Ferrell, James Caan, Bob Newhart, Zooey D...","300,546",$173.40M,https://m.media-amazon.com/images/M/MV5BMzUxNz...,Movie
4,How the Grinch Stole Christmas,PG,104.0,6.3,46.0,"Comedy, Family, Fantasy",2000.0,"On the outskirts of Whoville lives a green, re...",Ron Howard,"Jim Carrey, Taylor Momsen, Kelley, Jeffrey Tam...","280,898",$260.04M,https://m.media-amazon.com/images/M/MV5BNWNiNT...,Movie


In [ ]:
def exploracion(df):
    df_info = pd.DataFrame()
    df_info["% nulos"] = round(df.isna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["% no_nulos"] = round(df.notna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["tipo_dato"] = df.dtypes
    df_info["num_valores_unicos"] = df.nunique()
    print(f"""El DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.
Tiene {df.duplicated().sum()} datos duplicados, lo que supone un porcentaje de {round(df.duplicated().sum()/df.shape[0], 2)}% de los datos.
Hay {len(list(df_info[(df_info["% nulos"] != "0.0%")].index))} columnas con datos nulos, y son:
{list(df_info[(df_info["% nulos"] != "0.0%")].index)}
y sin nulos hay {len(list(df_info[(df_info["% nulos"] == "0.0%")].index))} columnas y son:
{list(df_info[(df_info["% nulos"] == "0.0%")].index)}
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:""")
    display(df_info.head())
    print("Principales estadísticos de las columnas categóricas:")
    display(df.describe(include="O").T)
    print("Principales estadísticos de las columnas numéricas:")
    display(df.describe(exclude="O").T)
    return df_info

exploracion(df)

El DataFrame tiene 873 filas y 14 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.
Hay 10 columnas con datos nulos, y son:
['rating', 'runtime', 'imdb_rating', 'meta_score', 'genre', 'release_year', 'director', 'stars', 'votes', 'gross']
y sin nulos hay 4 columnas y son:
['title', 'description', 'img_src', 'type']
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
title,0.0%,100.0%,object,848
rating,24.4%,75.6%,object,13
runtime,4.7%,95.3%,float64,95
imdb_rating,3.89%,96.11%,float64,61
meta_score,88.55%,11.45%,float64,54


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
title,873,848,A Christmas Carol,6
rating,660,13,TV-G,262
genre,872,126,"Comedy, Drama, Romance",109
description,873,858,Add a Plot,16
director,868,546,Peter Sullivan,13
stars,862,858,"Rose McIver, Ben Lamb, Alice Krige, Honor Knea...",3
votes,839,763,992,4
gross,79,79,$16.60M,1
img_src,873,854,https://m.media-amazon.com/images/S/sash/i-t32...,20
type,873,2,Movie,864


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
runtime,832.0,86.596154,18.152062,1.0,84.00,87.0,91.0,199.0
imdb_rating,839.0,6.100477,0.966392,1.3,5.60,6.2,6.7,9.2
meta_score,100.0,57.650000,17.901174,18.0,46.75,56.0,69.0,96.0
release_year,862.0,2009.498840,20.274696,1898.0,2010.00,2017.0,2020.0,2023.0


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
title,0.0%,100.0%,object,848
rating,24.4%,75.6%,object,13
runtime,4.7%,95.3%,float64,95
imdb_rating,3.89%,96.11%,float64,61
meta_score,88.55%,11.45%,float64,54
genre,0.11%,99.89%,object,126
release_year,1.26%,98.74%,float64,82
description,0.0%,100.0%,object,858
director,0.57%,99.43%,object,546
stars,1.26%,98.74%,object,858


In [7]:
# Informacion general 
print(f'Informacion df: {df.info()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         873 non-null    object 
 1   rating        660 non-null    object 
 2   runtime       832 non-null    float64
 3   imdb_rating   839 non-null    float64
 4   meta_score    100 non-null    float64
 5   genre         872 non-null    object 
 6   release_year  862 non-null    float64
 7   description   873 non-null    object 
 8   director      868 non-null    object 
 9   stars         862 non-null    object 
 10  votes         839 non-null    object 
 11  gross         79 non-null     object 
 12  img_src       873 non-null    object 
 13  type          873 non-null    object 
dtypes: float64(4), object(10)
memory usage: 95.6+ KB
Informacion df: None


# DUPLICADOS

In [ ]:
def get_duplicate_rows(df):
    """
    Encuentra y devuelve las filas completas duplicadas de un DataFrame.

    Args:
        df (pd.DataFrame): El DataFrame de entrada.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con las filas duplicadas completas.
    """
    # Identificar duplicados basados en todas las columnas
    duplicate_rows = df[df.duplicated(keep=False)]
    return duplicate_rows

get_duplicate_rows(df)